**Importações**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import colorsys
import io
from ipywidgets import widgets, HBox, VBox
from IPython.display import display
from PIL import Image

**Aplicação**

In [ ]:
class Coloras:
  def __init__(self,image_path):
    self.image_path = image_path
  # Faz uma máscara da roupa da imagem original
  def pred_mask(self):
    self.image_size = 256
    model = load_model('/content/drive/Shareddrives/RAS_Colors/Modelo - DeepLab/deep_lab_final.h5')
    model.load_weights('/content/drive/Shareddrives/RAS_Colors/Pesos - DeepLab/Pesos - Deep Lab/best_model_deep (7).h5')
    image = tf.io.read_file(self.image_path)
    image = tf.image.decode_png(image, channels=3)
    image.set_shape([None, None, 3])
    image = tf.image.resize(images=image, size=[self.image_size, self.image_size])
    image = image / 127.5 - 1
    self.prediction = model.predict(np.expand_dims((image), axis=0))
    self.prediction = np.squeeze(self.prediction)
    self.prediction = np.argmax(self.prediction, axis=2)
    return self.prediction
  # Recorta a roupa da imagem original com base na máscara
  def crop_img(self):
    self.img = cv2.imread(self.image_path)
    try:
      mask = self.prediction.astype(np.uint8)
      self.img = cv2.cvtColor(self.img,cv2.COLOR_BGR2RGB)
      self.img = cv2.resize(self.img,(self.image_size,self.image_size))
      mask = np.expand_dims(mask, axis=-1)
      ret, mask = cv2.threshold(mask, 10, 255, cv2.THRESH_BINARY)
      mask_inv = cv2.bitwise_not(mask)
      self.img_crop = cv2.bitwise_and(self.img,self.img,mask = mask_inv)
      return self.img_crop
    except AttributeError:
      print('Por favor use a função pred_mask antes de usar a crop_img')
  # Mostra as duas cores dominantes da roupa
  def two_predominant_colors(self):
    img = self.img_crop
    img = cv2.blur(img,(80,4))
    (r,g,b) = img[90,120] #pega o pixel
    img [88:92, 118:122] = (255,255,255) # coloca o ponto
    (r1,g1,b1) = img[173,123]
    img[170:175, 100:105] = (255,0,255)
    self.cor1_rgb = [r,g,b]
    self.cor2_rgb = [r1,g1,b1]
    r,g,b = r/255,g/255,b/255
    r1,g1,b1 = r1/255,g1/255,b1/255
    self.hsv = colorsys.rgb_to_hsv(r,g,b)
    self.hsv2 = colorsys.rgb_to_hsv(r1,g1,b1)
    h1 = self.hsv[0]
    h2 = self.hsv2[0]
    self.colors = [[h1,h2]]
    return self.colors
  def score(self):
    self.model_score = load_model('/content/drive/Shareddrives/RAS_Colors/Score/score (1).h5')
    self.model_score.load_weights('/content/drive/Shareddrives/RAS_Colors/Score/pesos (1).h5')
    self.scoring = self.model_score.predict(self.colors)
    return self.scoring
    

**Criando a interface**

In [ ]:
uploader = widgets.FileUpload(accept='.png',description = 'Escolha uma foto')
botao = VBox([uploader])
inputs = HBox([uploader])

def scoring(sender):
  for name,file_info in uploader.value.items():
    name = name
  app = Coloras(name)
  app.pred_mask()
  app.crop_img()
  app.two_predominant_colors()
  score = app.score()
  if score > 3:
    print( "\033[0;32m" +  '         {:.1f}'.format(score[0][0]))
  else:
    print( "\033[1;31m" +  '         {:.1f}'.format(score[0][0]))

  
botao = widgets.Button(description="Clique  para ver a nota")
botao.on_click(scoring)


In [ ]:
display(inputs,botao)

Button(description='Clique  para ver a nota', style=ButtonStyle())

         3.2
